# Carregar dados

In [19]:
import pandas as pd

# # Mostrar mais colunas
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', 100)

df = pd.read_parquet("./output/silver/rais_TODOS_ANOS_RAIS_VINC_PUB.parquet.gzip")


qnt_full = len(df)
print(f"""
Quantidade docs: {qnt_full}
""")

df.head(2)



Quantidade docs: 1543009



,ano,id_uf_municipio,remuneracao_media,desligamento_id,idade,ocupacao_id,sexo_id,sigla_uf,sigla_municipio,valor_remuneracao_media,raca_cor_id
0,2019,500240,997.99,12.0,17,317210,1,MS,Caarapo,NaN,NaN
1,2019,500570,0.00,11.0,36,212420,1,MS,Navirai,NaN,NaN


In [20]:
df[df["remuneracao_media"] > 0]

,ano,id_uf_municipio,remuneracao_media,desligamento_id,idade,ocupacao_id,sexo_id,sigla_uf,sigla_municipio,valor_remuneracao_media,raca_cor_id
0,2019,500240,997.99,12.0,17,317210,1,MS,Caarapo,NaN,NaN
2,2019,500270,1547.16,21.0,21,317110,1,MS,Campo Grande,NaN,NaN
3,2019,500270,1456.14,11.0,32,317205,1,MS,Campo Grande,NaN,NaN
4,2019,500270,1314.90,11.0,24,317205,1,MS,Campo Grande,NaN,NaN
5,2019,500295,1689.91,11.0,24,317210,1,MS,Chapadao do Sul,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
384170,2020,431490,5476.49,0.0,45,212410,2,RS,Porto Alegre,NaN,NaN
384171,2020,430860,2258.49,0.0,38,212420,2,RS,Garibaldi,NaN,NaN
384172,2020,431410,6300.00,0.0,25,317110,2,RS,Passo Fundo,NaN,NaN
384173,2020,431490,4161.69,0.0,28,212410,2,RS,Porto Alegre,NaN,NaN


In [2]:
df_sexo = pd.read_csv("./output/bronze/planilhas/dic_sexo.csv")
df_ordered_sinonimo = pd.read_csv("./output/bronze/planilhas/ordered_sinonimo.csv")

In [3]:
# change column ocupacao_id to int
df["ocupacao_id"] = df["ocupacao_id"].astype(int)

# Juntando tabelas

In [4]:

df = pd.merge(df, df_sexo, left_on="sexo_id", right_on="chave",
              how="left")
df = pd.merge(df, df_ordered_sinonimo, left_on="ocupacao_id",
              right_on="cbo_2002", how="left")
df.head(2)

,ano,id_uf_municipio,remuneracao_media,desligamento_id,idade,ocupacao_id,sexo_id,sigla_uf,sigla_municipio,valor_remuneracao_media,raca_cor_id,chave,valor,cbo_2002,sinonimo
0,2019,500240,997.99,12.0,17,317210,1,MS,Caarapo,NaN,NaN,1,Masculino,317210,Tecnico de Apoio ao Usuario de Informatica (Helpdesk)
1,2019,500570,0.00,11.0,36,212420,1,MS,Navirai,NaN,NaN,1,Masculino,212420,Analista de Suporte Computacional


### Alterando remuneração média de NaN para 0

In [5]:
df["remuneracao_media"].fillna(0, inplace=True)


# Gerando ID para Sigla_UF

In [6]:
# Create id for sigla_uf
df["sigla_uf_id"] = df["sigla_uf"].astype("category").cat.codes
df.head(2)


,ano,id_uf_municipio,remuneracao_media,desligamento_id,idade,ocupacao_id,sexo_id,sigla_uf,sigla_municipio,valor_remuneracao_media,raca_cor_id,chave,valor,cbo_2002,sinonimo,sigla_uf_id
0,2019,500240,997.99,12.0,17,317210,1,MS,Caarapo,NaN,NaN,1,Masculino,317210,Tecnico de Apoio ao Usuario de Informatica (Helpdesk),11
1,2019,500570,0.00,11.0,36,212420,1,MS,Navirai,NaN,NaN,1,Masculino,212420,Analista de Suporte Computacional,11


## Gerando Regiões por Sigla

In [7]:
# SEPARATE sigla_uf by regions

# https://pt.wikipedia.org/wiki/Regi%C3%B5es_do_Brasil
# https://pt.wikipedia.org/wiki/Unidades_federativas_do_Brasil
REGIOES = {
    "Norte": ["AC", "AP", "AM", "PA", "RO", "RR", "TO"],
    "Nordeste": ["AL", "BA", "CE", "MA", "PB", "PE", "PI", "RN", "SE"],
    "Centro-Oeste": ["GO", "MT", "MS", "DF"],
    "Sudeste": ["ES", "MG", "RJ", "SP"],
    "Sul": ["PR", "RS", "SC"]
}


In [8]:
df_regiao = df.copy()

# add col Regioes to df_ana with REGIOES
for regiao, siglas in REGIOES.items():
    df_regiao.loc[df["sigla_uf"].isin(
        siglas), "regiao"] = regiao

df_regiao["regiao_id"] = df_regiao["regiao"].astype("category").cat.codes
df = df_regiao
df.head(2)


,ano,id_uf_municipio,remuneracao_media,desligamento_id,idade,ocupacao_id,sexo_id,sigla_uf,sigla_municipio,valor_remuneracao_media,raca_cor_id,chave,valor,cbo_2002,sinonimo,sigla_uf_id,regiao,regiao_id
0,2019,500240,997.99,12.0,17,317210,1,MS,Caarapo,NaN,NaN,1,Masculino,317210,Tecnico de Apoio ao Usuario de Informatica (Helpdesk),11,Centro-Oeste,0
1,2019,500570,0.00,11.0,36,212420,1,MS,Navirai,NaN,NaN,1,Masculino,212420,Analista de Suporte Computacional,11,Centro-Oeste,0


# Desligamento

In [9]:
# Create new col desligamento from desligament_id, if != 0, so "nao", otherwise "sim"
df["desligamento"] = df["desligamento_id"].apply(
    lambda x: 0 if x != 0 else 1)
df.head(2)



,ano,id_uf_municipio,remuneracao_media,desligamento_id,idade,ocupacao_id,sexo_id,sigla_uf,sigla_municipio,valor_remuneracao_media,raca_cor_id,chave,valor,cbo_2002,sinonimo,sigla_uf_id,regiao,regiao_id,desligamento
0,2019,500240,997.99,12.0,17,317210,1,MS,Caarapo,NaN,NaN,1,Masculino,317210,Tecnico de Apoio ao Usuario de Informatica (Helpdesk),11,Centro-Oeste,0,0
1,2019,500570,0.00,11.0,36,212420,1,MS,Navirai,NaN,NaN,1,Masculino,212420,Analista de Suporte Computacional,11,Centro-Oeste,0,0


# Selecionando colunas necessárias

In [10]:
df = df[["ano", "sigla_uf_id", "regiao", "regiao_id", "sigla_uf", "remuneracao_media", 'desligamento', "idade",
         "ocupacao_id", "sinonimo", "sexo_id", "valor"]]

df = df.rename(columns={
    "sinonimo": "ocupacao",
    "valor": "sexo"
    })

df.head(2)

,ano,sigla_uf_id,regiao,regiao_id,sigla_uf,remuneracao_media,desligamento,idade,ocupacao_id,ocupacao,sexo_id,sexo
0,2019,11,Centro-Oeste,0,MS,997.99,0,17,317210,Tecnico de Apoio ao Usuario de Informatica (Helpdesk),1,Masculino
1,2019,11,Centro-Oeste,0,MS,0.00,0,36,212420,Analista de Suporte Computacional,1,Masculino


# Verificando se tem mais algum valor Nulo

In [11]:

df[df.isna().any(axis=1)]


,ano,sigla_uf_id,regiao,regiao_id,sigla_uf,remuneracao_media,desligamento,idade,ocupacao_id,ocupacao,sexo_id,sexo


# Colocando a idade com o mínimo

In [17]:
df.loc[df["idade"] == 0, "idade"] = 14
df["idade"].describe()


count    1.543009e+06
mean     3.396295e+01
std      9.633748e+00
min      1.400000e+01
25%      2.700000e+01
50%      3.200000e+01
75%      3.900000e+01
max      9.200000e+01
Name: idade, dtype: float64

# Salvando arquivo

In [18]:
df.to_parquet("./output/gold/rais_TODOS_ANOS_comJoin_RAIS_VINC_PUB.parquet.gzip", index=False, compression="gzip")